# Installing required packages

In [ ]:
!pip install --upgrade accelerate
!pip install datasets
!pip install transformers==4.27.0
!pip install evaluate seqeval

In [ ]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, Features, Value, ClassLabel, Sequence, load_dataset
import evaluate
from seqeval.metrics import classification_report
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline
import warnings

from google.colab import drive
drive.mount('/content/drive')

# Processing source file

In [ ]:
mga = pd.read_excel('/content/mga_ner.xlsx')
mga['split_sent'] = mga['split_sent'].apply(lambda x: x.replace("'", ''))
mga['split_sent'] = mga['split_sent'].apply(lambda x: x.replace("[", ''))
mga['split_sent'] = mga['split_sent'].apply(lambda x: x.replace("]", ''))
mga['split_sent'] = mga['split_sent'].apply(lambda x: x.replace(",", ''))
mga['ner'] = mga['ner'].apply(lambda x: x.replace("]", ''))
mga['ner'] = mga['ner'].apply(lambda x: x.replace("[", ''))
mga['ner'] = mga['ner'].apply(lambda x: x.replace("'", ''))
mga['ner'] = mga['ner'].apply(lambda x: x.replace(",", ''))
mga['split_split_sent'] = mga['split_sent'].apply(lambda x: x.split())
mga['split_ner'] = mga['ner'].apply(lambda x: x.split())

In [ ]:
def to_json(x, y):
  global mga_json
  mga_json.append({'sentence': x, 'tags': y})

mga_json = []
mga.apply(lambda x: to_json(x['split_split_sent'], x['split_ner']), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
1439    None
1440    None
1441    None
1442    None
1443    None
Length: 1444, dtype: object

In [ ]:
ds_IOB2 = pd.DataFrame(data_all, columns = ['tokens', 'ner_tags_labels'])

In [ ]:
tag_ner = []
for d in data_all:
  for t in d['tags']:
    if t not in tag_ner:
      tag_ner.append(t)

In [ ]:
tags_clean = []

for d in data_all:
  new_tags = []
  for i in range(len(d['tags'])):
    if d['tags'][i] == 'B-MORH':
      new_tags.append('B-MORPH')
    elif d['tags'][i] == 'U-SYNT':
      new_tags.append('I-SYNT')
    elif d['tags'][i] == 'D-MORPH':
      new_tags.append('B-MORPH')
    else:
      new_tags.append(d['tags'][i])
  tags_clean.append({'sentence': d['sentence'], 'tags': new_tags})
    
ds_IOB2 = pd.DataFrame(tags_clean, columns = ['sentence', 'tags'])

In [ ]:
ner_tags_labels_ = []
for d in tags_clean:
  for t in d['tags']:
    if t not in ner_tags_labels_:
      ner_tags_labels_.append(t)
print(ner_tags_labels_)

['O', 'B-MORPH', 'B-PHON', 'B-LEX', 'B-SYNT', 'I-SYNT']


In [ ]:
ner_tags_labels_ = ['O', 'B-PHON', 'B-MORPH', 'I-MORPH', 'B-LEX', 'I-LEX', 'B-SYNT', 'I-SYNT']

# Create dataset from our data

In [ ]:
ds_features = Features({'sentence':  Sequence(Value("string")),
                        'tags': Sequence(ClassLabel(names=ner_tags_labels_))})

dataset = Dataset.from_pandas(ds_IOB2, features=ds_features)
dataset_splitted = dataset.train_test_split(test_size=0.3, seed=88)

# Define the model

In [ ]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Defining a function that transforms the input

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["sentence"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids: 
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
ds_tokenized = dataset_splitted.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1010 [00:00<?, ? examples/s]

Map:   0%|          | 0/434 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Prepare function for evaluating

In [ ]:
seqeval = evaluate.load("seqeval")
label_list = ds_tokenized["train"].features["tags"].feature.names

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
id2label = {i: label_list[i] for i in range(len(label_list))}
label2id = {label_list[i]: i for i in range(len(label_list))}

# Load the model 

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list), id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-st

In [ ]:
warnings.filterwarnings('ignore')

# Train

In [ ]:
training_args = TrainingArguments(
    output_dir="xlm_roberta_base_dial",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=50,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_tokenized["train"],
    eval_dataset=ds_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.389300,0.346516,0.000000,0.000000,0.000000,0.915325
1000,0.255700,0.331808,0.520000,0.220339,0.309524,0.917425
1500,0.181400,0.371128,0.381679,0.423729,0.401606,0.899930
2000,0.129500,0.433363,0.482759,0.355932,0.409756,0.916725
2500,0.093400,0.490739,0.411215,0.372881,0.391111,0.906928
3000,0.065900,0.575736,0.560000,0.355932,0.435233,0.923723
3500,0.051100,0.582593,0.500000,0.372881,0.427184,0.917425
4000,0.036600,0.619856,0.460000,0.389831,0.422018,0.913226
4500,0.033700,0.676284,0.557143,0.330508,0.414894,0.924423
5000,0.023000,0.697041,0.567164,0.322034,0.410811,0.923023


TrainOutput(global_step=6350, training_loss=0.10295498735322726, metrics={'train_runtime': 1121.0766, 'train_samples_per_second': 45.046, 'train_steps_per_second': 5.664, 'total_flos': 361065358086048.0, 'train_loss': 0.10295498735322726, 'epoch': 50.0})

# Evaluate

In [ ]:
best_model_from_training_testing = '/content/xlm_roberta_base_dial/checkpoint-3000'
best_model= AutoModelForTokenClassification.from_pretrained(best_model_from_training_testing, num_labels=len(label_list), id2label=id2label, label2id=label2id)
trainer = Trainer(best_model, tokenizer=tokenizer, data_collator=data_collator)
output = trainer.predict(ds_tokenized['test'].remove_columns(['sentence', 'tags']))
predictions, labels, metrics = output
predictions = np.argmax(predictions, axis=2)

true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
print(classification_report(true_labels, true_predictions))

              precision    recall  f1-score   support

         LEX       0.00      0.00      0.00         3
       MORPH       0.67      0.47      0.55        55
        PHON       0.46      0.28      0.35        57
        SYNT       0.00      0.00      0.00         3

   micro avg       0.56      0.36      0.44       118
   macro avg       0.28      0.19      0.23       118
weighted avg       0.53      0.36      0.43       118



# Creating a pipeline that will accept transcriptions and generate a list of tags for each token

In [ ]:
class MyPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        if "tokenizer" in kwargs:
            preprocess_kwargs["tokenizer"] = kwargs["tokenizer"]
        return preprocess_kwargs, {}, {}

    def preprocess(self, text):
        self.text_splt = wordpunct_tokenize(text.lower())
        self.tokenized = self.tokenizer(self.text_splt, is_split_into_words=True, return_tensors='pt')
        return self.tokenized

    def _forward(self, model_inputs):
        model_inputs['input_ids'] = model_inputs['input_ids'].to('cuda')
        model_inputs['attention_mask'] = model_inputs['attention_mask'].to('cuda')
        return self.model(**model_inputs)

    def postprocess(self, model_outputs):
        tokens = self.tokenizer.convert_ids_to_tokens(list(self.tokenized["input_ids"][0]))
        predicted_label_id = torch.argmax(model_outputs.logits, axis=-1).numpy()
        labels = [id2label[i] for i in predicted_label_id[0]]
        res = {'tokens': tokens, 'labels': labels}
        result_text = ''
        result_labels = ''
        for i in range(len(res['tokens'])):
            if res['tokens'][i] != '<s>' and res['tokens'][i] != '</s>':
                if res['tokens'][i].startswith('▁'):
                    res['labels'][i] = '▁' + res['labels'][i]
                if i > 1:
                    x = res['tokens'][i].replace('▁', ' ')
                    y = res['labels'][i].replace('▁', ' ')
                else:
                    x = res['tokens'][i].replace('▁', '')
                    y = res['labels'][i].replace('▁', '')
                result_text += x
                result_labels = result_labels + '|' + y
        result_labels_splt = result_labels.split(' ')
        final_labels = []
        for l in result_labels_splt:
            cnt = Counter()
            if l[0] == '|' and l[-1] == '|':
                l = l[1:-1]
            elif l[0] != '|' and l[-1] == '|':
                l = l[0:-1]
            elif l[0] == '|' and l[-1] != '|':
                l = l[1:]
            l_splt = l.split('|')
            if len(l_splt) == 1:
                res_lab = l_splt[0]
                final_labels.append(res_lab)
            if len(l_splt) > 1:
                cnt = Counter(l_splt)
                if len(dict(cnt)) == 1:
                    res_lab = l_splt[0]
                    final_labels.append(res_lab)
                else:
                    c = dict(cnt)
                    c.pop('O', None)
                    res_lab = max(c, key=c.get)
                    final_labels.append(res_lab)

        txt_splt = result_text.split(' ')
        dict_with_labels = {}
        for i in range(len(txt_splt)):
            dict_with_labels[i] = {txt_splt[i]: final_labels[i]}

        return dict_with_labels

In [ ]:
pipeline = MyPipeline(model=model.to('cuda'), tokenizer=tokenizer)
labels = []
for sentence in tqdm(list(mga['sentence'])):
    res = pipeline(sentence)
    labels_sent = []
    for key, value in res.items():
        for key2, value2 in value.items():
            labels_sent.append(value2)
    labels.append(labels_sent) 

100%|██████████| 1444/1444 [00:22<00:00, 65.07it/s]


In [ ]:
mga['predictions'] = labels

In [ ]:
mga.to_excel('mga_ner_predictions.xlsx')  

# Examples

In [ ]:
pipeline('говорит это ну папка был белобилетник у него была одна короче')

{0: {'говорит': 'B-MORPH'},
 1: {'это': 'O'},
 2: {'ну': 'O'},
 3: {'папка': 'O'},
 4: {'был': 'O'},
 5: {'белобилетник': 'O'},
 6: {'у': 'O'},
 7: {'него': 'O'},
 8: {'была': 'O'},
 9: {'одна': 'O'},
 10: {'короче': 'O'}}

In [ ]:
pipeline('наляпивши')

{0: {'наляпивши': 'B-MORPH'}}

In [ ]:
pipeline('вот тут где вот')

{0: {'вот': 'O'}, 1: {'тут': 'B-MORPH'}, 2: {'где': 'O'}, 3: {'вот': 'O'}}

In [ ]:
pipeline('на тракториста выучилась')

{0: {'на': 'O'}, 1: {'тракториста': 'O'}, 2: {'выучилась': 'B-MORPH'}}

# Save model to disk

In [ ]:
!zip -r /content/drive/MyDrive/model_roberta.zip /content/xlm_roberta_base_dial/checkpoint-3000

  adding: content/xlm_roberta_base_dial/checkpoint-3000/ (stored 0%)
  adding: content/xlm_roberta_base_dial/checkpoint-3000/tokenizer.json (deflated 76%)
  adding: content/xlm_roberta_base_dial/checkpoint-3000/rng_state.pth (deflated 28%)
  adding: content/xlm_roberta_base_dial/checkpoint-3000/scheduler.pt (deflated 48%)
  adding: content/xlm_roberta_base_dial/checkpoint-3000/optimizer.pt (deflated 71%)
  adding: content/xlm_roberta_base_dial/checkpoint-3000/special_tokens_map.json (deflated 52%)
  adding: content/xlm_roberta_base_dial/checkpoint-3000/tokenizer_config.json (deflated 49%)
  adding: content/xlm_roberta_base_dial/checkpoint-3000/trainer_state.json (deflated 73%)
  adding: content/xlm_roberta_base_dial/checkpoint-3000/config.json (deflated 53%)
  adding: content/xlm_roberta_base_dial/checkpoint-3000/training_args.bin (deflated 48%)
  adding: content/xlm_roberta_base_dial/checkpoint-3000/pytorch_model.bin (deflated 29%)
